[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/CMU-IDeeL/CMU-IDeeL.github.io/blob/master/F25/document/Recitation_0_Series/0.19/0_19_Losses_Part_1.ipynb)

# Recitation 0: Losses

Prepared by: Massa Baali (mbaali@andrew.cmu.edu)


**Where do we use the loss? **

In [ ]:
# Template for calling the loss
# Training loop
n_epochs = 5
for epoch in range(n_epochs):
    model.train()
    # Loop through your dataset
    for batch in dataloader:
        # Forward pass
        outputs = model(batch)
        optimizer.zero_grad()

        # Forward pass
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()


### Mean Squared Error (MSE)


In [ ]:
import torch
import torch.nn as nn

y_true = torch.tensor([300.0, 400.0, 500.0])  # Ground truth (Real Prices)
y_pred = torch.tensor([280.0, 390.0, 520.0])  # Predictions  (Predicted Prices)

mse_loss = nn.MSELoss()
loss = mse_loss(y_pred, y_true)

print("Mean Squared Error:", loss.item()) # the lower the value the better the model


Mean Squared Error: 300.0


### Cross Entropy (CE)


In [ ]:
import torch
import torch.nn as nn

# True labels (class indices)
y_true = torch.tensor([0, 1, 2])  # 3 examples: Cat, Dog, Bird

# Predicted probabilities (logits for 3 classes: Cat, Dog, Bird)
y_pred = torch.tensor([[0.7, 0.2, 0.1],  # Probabilities for example 1
                       [0.1, 0.8, 0.1],  # Probabilities for example 2
                       [0.2, 0.3, 0.5]])  # Probabilities for example 3

# Use CrossEntropyLoss
criterion = nn.CrossEntropyLoss()
loss = criterion(y_pred, y_true)

print("Cross-Entropy Loss:", loss.item())


Cross-Entropy Loss: 0.7991690635681152


### Binary Cross Entropy (BCE)


In [ ]:
import torch
import torch.nn as nn

# True labels (1: spam, 0: not spam)
y_true = torch.tensor([1.0, 0.0, 1.0])  # 3 examples

# Predicted probabilities (model's confidence)
y_pred = torch.tensor([0.85, 0.1, 0.95])

# Use Binary Cross-Entropy Loss
bce_loss = nn.BCELoss()
loss = bce_loss(y_pred, y_true)

print("Binary Cross-Entropy Loss:", loss.item())


Binary Cross-Entropy Loss: 0.10639091581106186


### Centerloss


In [ ]:
import torch
import torch.nn as nn

class CenterLoss(nn.Module):
    def __init__(self, num_classes=10, num_features=2):
        super(CenterLoss, self).__init__()
        self.num_class = num_classes
        self.num_feature = num_features
        self.centers = nn.Parameter(torch.randn(self.num_class, self.num_feature))

    def forward(self, x, labels):
        # Get class centers for the input labels
        center = self.centers[labels]
        # Compute squared distance between features and centers
        dist = (x-center).pow(2).sum(dim=-1)
        # Distances are clamped to ensure they stay within a reasonable range, and the mean distance is computed to get the final loss.
        loss = torch.clamp(dist, min=1e-12, max=1e+12).mean(dim=-1)
        return loss

embeddings = torch.tensor([[1.0, 1.0], [2.0, 2.0], [0.5, 0.5]])  # Embedding vectors
labels = torch.tensor([0, 1, 2])  # Corresponding class labels
CenterLoss  = CenterLoss(num_classes=3, num_features=2)
loss = CenterLoss(embeddings, labels)
print("Center Loss:", loss.item())



Center Loss: 3.6008317470550537


### Angular Softmax


In [ ]:
import torch
import torch.nn as nn


class AMSoftmax(nn.Module):

    def __init__(self,
                 in_feats,
                 n_classes=10,
                 m=0.3,
                 s=15):
        super(AMSoftmax, self).__init__()
        self.m = m
        self.s = s
        self.in_feats = in_feats
        self.W = torch.nn.Parameter(torch.randn(in_feats, n_classes), requires_grad=True)
        self.ce = nn.CrossEntropyLoss()
        nn.init.xavier_normal_(self.W, gain=1)

    def forward(self, x, lb):
        assert x.size()[0] == lb.size()[0]
        assert x.size()[1] == self.in_feats
        # Normalization
        x_norm = torch.norm(x, p=2, dim=1, keepdim=True).clamp(min=1e-9)
        x_norm = torch.div(x, x_norm)
        w_norm = torch.norm(self.W, p=2, dim=0, keepdim=True).clamp(min=1e-9)
        w_norm = torch.div(self.W, w_norm)
        # compute cos sim
        costh = torch.mm(x_norm, w_norm)
        delt_costh = torch.zeros_like(costh).scatter_(1, lb.unsqueeze(1), self.m)
        #subtracts the angular margin from the cos sim for the true clas
        costh_m = costh - delt_costh
        # scaled by the factor
        costh_m_s = self.s * costh_m
        loss = self.ce(costh_m_s, lb)
        return loss


# Inputs: embeddings and labels
embeddings = torch.tensor([[0.8, 0.6], [0.6, 0.8]])  # 2D embeddings
labels = torch.tensor([0, 1])  # Class labels

loss = AMSoftmax(in_feats=2, n_classes=2, m=0.35, s=30)(embeddings, labels)
print("Angular SoftMax Loss:", loss.item())


Angular SoftMax Loss: 18.93401336669922


### Additive Margin Softmax

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class AAMSoftmaxLoss(nn.Module):

    def __init__(self, embedding_dim, n_classes, scale = 30.0, margin=0.4):
        super(AAMSoftmaxLoss, self).__init__()
        self.scale = scale
        self.margin = margin
        self.embedding_dim = embedding_dim

        self.n_classes = n_classes
        self.W = torch.nn.Parameter(torch.randn(embedding_dim, n_classes), requires_grad=True)

        self.ce = nn.CrossEntropyLoss()

    def forward(self, x, labels):
        '''
        Input shape (N, embedding_dim)
        '''
        n, m = x.shape
        assert n == len(labels)
        assert m == self.embedding_dim

        # Normalization
        x_norm = torch.norm(x, p=2, dim=1, keepdim=True).clamp(min=1e-9)
        x_norm = torch.div(x, x_norm)
        w_norm = torch.norm(self.W, p=2, dim=0, keepdim=True).clamp(min=1e-9)
        w_norm = torch.div(self.W, w_norm)
        # cos sim
        cos_theta = torch.matmul(w_norm, x_norm.T).T
        # subtracts the angular margin from the cosine similarity
        psi = cos_theta - self.margin

        onehot = F.one_hot(labels, self.n_classes)
        # The modified cosine sim (psi) is scaled by the factor scale
        logits = self.scale * torch.where(onehot == 1, psi, cos_theta)
        err = self.ce(logits, labels)

        return err

# Inputs: embeddings and labels
embeddings = torch.tensor([[0.8, 0.6], [0.6, 0.8]])  # 2D embeddings
labels = torch.tensor([0, 1])  # Class labels

loss = AAMSoftmaxLoss(embedding_dim=2, n_classes=2, scale=30, margin=0.35)(embeddings, labels)
print("Additive Angular Margin SoftMax Loss:", loss.item())


Additive Angular Margin SoftMax Loss: 29.59213638305664


### Triplet Loss

In [ ]:
import torch
import torch.nn.functional as F

# Example embeddings for anchor, positive, and negative
anchor = torch.tensor([1.0, 2.0])
positive = torch.tensor([1.2, 2.1])  # Same class as anchor
negative = torch.tensor([3.0, 4.0])  # Different class from anchor

margin = 1.0  # Triplet margin
def triplet(anchor, positive, negative, margin):
  # Compute distances
  pos_dist = F.pairwise_distance(anchor.unsqueeze(0), positive.unsqueeze(0))
  neg_dist = F.pairwise_distance(anchor.unsqueeze(0), negative.unsqueeze(0))
  # Compute triplet loss (apply relu to ensure that the loss is non-negative)
  loss = torch.relu(pos_dist - neg_dist + margin)
  return loss

loss = triplet(anchor, positive, negative, margin)

print("Triplet Loss:", loss.item())


Triplet Loss: 0.0


# References and further reading

1.   https://github.com/tomastokar/Additive-Margin-Softmax/blob/main/AMSloss.py
2.   https://github.com/CoinCheung/pytorch-loss/tree/master/pytorch_loss
3.   https://pytorch.org/docs/stable/generated/torch.nn.TripletMarginLoss.html
4.   https://www.kaggle.com/code/farhanrenaldi/pytorch-faster-rcnn-w-center-loss-for-fmd#Center-Loss
5.   https://arxiv.org/abs/1801.05599
6.   https://arxiv.org/pdf/1806.03464
7.   https://kpzhang93.github.io/papers/eccv2016.pdf
8.   https://arxiv.org/pdf/1503.03832
